In [ ]:
import numpy as np
from qiskit import Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import plot_histogram
from qiskit.aqua import run_algorithm
from math import pi, acos, sqrt, sin
from qiskit.tools.visualization import circuit_drawer
from qiskit.aqua.components.iqfts import Standard
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.utils import CircuitFactory
from qiskit.aqua.circuits import PhaseEstimationCircuit
import time

%matplotlib inline

In [ ]:
def sprinkler(qc,q,a):
    qc.x(q[2])
    qc.x(a[0])
    qc.x(a[1])
    qc.x(a[2])
    qc.ccx(q[0],q[1],a[0])
    qc.ccx(q[1],q[2],a[1])
    qc.ccx(q[0],q[2],a[2])
    qc.mct([a[0],a[1],a[2],q[3]],a[3])
    qc.ccx(q[0],q[2],a[2])
    qc.ccx(q[1],q[2],a[1])
    qc.ccx(q[0],q[1],a[0])
    qc.x(a[2])
    qc.x(a[1])
    qc.x(a[0])
    qc.x(q[2])
    


In [ ]:
def prepare():
    q=QuantumRegister(4)
    qc=QuantumCircuit(q)
    theta0=2.0*asin(sqrt(0.55))
    theta1=2.0*asin(sqrt(0.3))
    theta2=2.0*asin(sqrt(0.7))
    qc.ry(theta0,q[0])
    qc.ry(theta1,q[1])
    qc.ry(theta2,q[2])

    qc.h(q[3])
    return qc

prep=prepare().to_gate()
invprep=prep.inverse()

In [ ]:
def grover():
    q=QuantumRegister(4)
    a=QuantumRegister(5)
    qc=QuantumCircuit(q,a)
    qc.x(a[3])
    qc.h(a[3])
    sprinkler(qc,q,a)
    qc.h(a[3])
    qc.x(a[3])
    qc.append(invprep,range(4))
    for i in range(q.size):
        qc.x(q[i])
    qc.mct([q[0],q[1],q[2],q[3]],a[4])
    qc.z(a[4])
    qc.mct([q[0],q[1],q[2],q[3]],a[4])
    for i in range(q.size):
        qc.x(q[i])
    qc.ry(2*pi,q[0]) 
    qc.append(prep,range(4))

    return qc

In [ ]:
ggrover = grover().to_gate()
cggrover = ggrover.control()

ggrover.label = "Grover"

In [ ]:
def qft(n):
    """Creates an n-qubit QFT circuit"""
    circuit = QuantumCircuit(n)
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit
    def qft_rotations(circuit, n):
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cu1(np.pi/2**(n-qubit), qubit, n)
        qft_rotations(circuit, n)
    
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit


In [ ]:
t = 7   # no. of counting qubits
n = 4   # no. of searching qubits

qft_dagger = qft(t).to_gate().inverse()
qft_dagger.label = "QFT†"



q=QuantumRegister(n)
a=QuantumRegister(5)
s=QuantumRegister(t)
c=ClassicalRegister(t)

qc = QuantumCircuit(s,q,a,c) # Circuit with n+t qubits and t classical bits

for qubit in range(t):
    qc.h(qubit)

qc.append(prep, range(t,n+t))

# Begin controlled Grover iterations
iterations = 1
for qubit in range(t):
    for i in range(iterations):
        qc.append(cggrover, [qubit] + [*range(t, n+t+5)])
    iterations *= 2
    
# Do inverse QFT on counting qubits
qc.append(qft_dagger, range(t))

# Measure counting qubits
qc.measure(range(t), range(t))

In [ ]:
from qiskit import IBMQ
IBMQ.save_account('TOKEN')

In [ ]:
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_16_melbourne')
print(backend, backend.status().pending_jobs)
start = time.time()

job_sim = execute(cc, backend, shots=10)
end = time.time()
print(end - start)

result_sim = job_sim.result()

In [ ]:
job_sim.error_message()

In [ ]:
counts  = result_sim.get_counts()
weights={}
angles={}
sint={}
sumof=0
for  key, value in counts.items():
    theta=2*pi*int(key,2)/2**t
    newkey=round(2*sin(theta/2)**2,3)
    if newkey in weights:
        weights[newkey]=weights[newkey]+value
    else:
        weights[newkey]=value

In [ ]:
weights

In [ ]:
fig=plot_histogram([weights],sort='asc',bar_labels=False, legend=['qiskit'])
fig